In [ ]:
# importing necessary netket libraries
import flax
import flax.linen as nn
import netket as nk
from netket.operator.spin import sigmax,sigmaz
import jax.numpy as jnp

In [ ]:
# other useful libraries
import pickle
import sys
import time

from tqdm import tqdm
from scipy.sparse.linalg import eigsh
from scipy.optimize import minimize

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# function to generate Hamiltonian and caclculate energy

def generate_Hamiltonian(N,J,h,hi):

  # single body terms
  H = sum([-J*sigmax(hi,i) for i in range(N)])
  # two body terms of the Hamiltonian
  H += sum([-h*sigmaz(hi,i)*sigmaz(hi,(i+1)%N) for i in range(N)])

  hstrings = H.to_pauli_strings()
  pauliop = list(hstrings.operators)
  coeff =  list(hstrings.weights)

  return H

def calc_Energy(H):
  sp_h = H.to_sparse()
  eig_vals,eig_vecs = eigsh(sp_h,k=2,which="SA") # selecting only 2 eigvals and eigvecs that having smallest algebraic eigenvalues
  Egs = eig_vals[0]
  gs_state = eig_vecs[0]

  return Egs

In [ ]:
# TFI model parameters
N = 15
J = 1
h = 1

# generating hamiltonian and theoretical energy
chain = nk.graph.Chain(N) # Periodic Boundary Conditions
hi = nk.hilbert.Spin(s=1/2) ** N
H = generate_Hamiltonian(N,J,h,hi)
Egs = calc_Energy(H)

In [ ]:
# NQS hyperparameters
alphas = [1,2,3,4]
niter = 300

In [ ]:

res = []


# Defining the variational state using RBM
for alpha in alphas:
  model = nk.models.RBM(alpha=alpha,param_dtype= complex) # parameter_dtype can be real also
  sampler = nk.sampler.MetropolisLocal(hi)
  vstate = nk.vqs.MCState(sampler,model,n_samples=10000) # number of samples taken is 10000
  np.random.seed(42)
  vstate.init_parameters() # initialising parameters
  log=nk.logging.RuntimeLog()
  optimizer = nk.optimizer.Sgd(learning_rate=0.1) # using stochastic gradient descent for optimization.
  gs = nk.driver.VMC(H,optimizer,variational_state=vstate,preconditioner=nk.optimizer.SR(holomorphic=True)) # build the optimization driver.
  st = time.time()
  gs.run(n_iter=niter,out=log)
  et = time.time()
  min,sec = divmod(et-st,60)
  data = log.data # log data.
  opt_energy = vstate.expect(H) # final energy expectation value.
  error = abs((opt_energy.mean-Egs)/Egs) # error in energy expectation value

  res_dict = {}

  res_dict['N'] = N
  res_dict['J'] = J
  res_dict['h'] = h
  res_dict['alpha'] = alpha
  res_dict['niter'] = niter
  res_dict['num_parameters'] = vstate.n_parameters
  res_dict['optmization_data'] = data['Energy'].to_dict()
  res_dict['relative_error'] = error
  res_dict['exec_time'] = f'{int(min)}:{int(sec)}'
  res.append(res_dict)
  print("optimized energy and relative error",opt_energy,error)

In [1]:
import netket
netket.__version__

'3.10.2'